# Trading Strategy R&D Assignment
### Joe Morris | JJM5298

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Import Data

Import monthly pricing data, annual financial data, and fama french factors

In [2]:
# monthly pricing
monthly = pd.read_csv('monthly.csv')

# annual R&D
annual = pd.read_csv('annual.csv')
# drop unnecessary columns
annual.drop(columns=['costat', 'indfmt', 'consol', 'popsrc', 'datafmt'], inplace=True)
# change datadate to datetime object
annual['datadate'] = pd.to_datetime(annual['datadate'])
# shift 90 days for look ahead bias
annual['datadate'] = annual['datadate'] + pd.Timedelta(days=90)

## Fama French 3 Factors

In [3]:
# fama french 3 factors
fama_french = pd.read_csv('F-F_Research_Data_Factors.csv')
# rename index to descriptive date
fama_french.rename({'Unnamed: 0': 'year_month'}, axis=1, inplace=True)
# change year_month to datetime object
fama_french['year_month'] = pd.to_datetime(fama_french['year_month'], format='%Y%m')
# get rid of days that come with datetime object
fama_french['year_month'] = fama_french['year_month'].dt.to_period('M')
# make the rf rate a decimal
fama_french['RF'] = fama_french['RF'] / 100

fama_french.head(2)

,year_month,Mkt-RF,SMB,HML,RF
0,1926-07,2.96,-2.56,-2.43,0.0022
1,1926-08,2.64,-1.17,3.82,0.0025


## Clean Monthly Data

In [4]:
# change date to datetime object
monthly['date'] = pd.to_datetime(monthly['date'])

# get rid of returns that are non numeric
monthly['RET'] = pd.to_numeric(monthly['RET'], errors='coerce')
# drop returns that are nan
monthly.dropna(subset=['RET'], inplace=True)
# remove returns less than -100%
monthly = monthly[monthly['RET'] > -1]
# sort values by date for easier viewing
monthly.sort_values('date', inplace=True)
# change prices to be all positive (Bid-Ask Spread filled values could make them negative)
monthly['PRC'] = monthly['PRC'].apply(lambda x: abs(x))
# drop unnecessary columns
monthly.drop(columns=['EXCHCD', 'SICCD', 'TICKER'], inplace=True)

# rename columns for better understanding
monthly.rename({'date': 'monthly_date', 'COMNAM': 'company_name', 'RET': 'monthly_return', 'SHROUT': 'shares_outstanding', 'PRC': 'monthly_price'}, axis=1, inplace=True)

monthly.head(2)


,PERMNO,monthly_date,company_name,monthly_price,monthly_return,shares_outstanding
978103,27829,1975-01-31,CROWNAMERICA INC,3.875,0.192308,1300.0
1124457,34841,1975-01-31,OGDEN CORP,17.625,0.317757,10090.0


### Change monthly returns to be in excess of the risk free rate

In [5]:
# add year and month columns to both data sets for merging
fama_french['year'] = fama_french['year_month'].dt.year
fama_french['month'] = fama_french['year_month'].dt.month
monthly['year'] = monthly['monthly_date'].dt.year
monthly['month'] = monthly['monthly_date'].dt.month

# merge monthly and fama french data to get the right RF rate for each month
monthly = pd.merge(monthly, fama_french, how='left', on=['year', 'month'])

# change the monthly_return by subtracting the rf rate (Excess Return)
monthly['monthly_return'] = monthly['monthly_return'] - monthly['RF']
# drop unnecessary columns
monthly.drop(columns=['year', 'month', 'year_month', 'RF', 'Mkt-RF', 'SMB',	'HML'], inplace=True)

monthly.head(2)

,PERMNO,monthly_date,company_name,monthly_price,monthly_return,shares_outstanding
0,27829,1975-01-31,CROWNAMERICA INC,3.875,0.186508,1300.0
1,34841,1975-01-31,OGDEN CORP,17.625,0.311957,10090.0


## Clean Annual Data

In [6]:
# filter by exchange 11-19
annual = annual[(annual['exchg'] >= 11) & (annual['exchg'] <= 19)]
# filter by sic not between 6000-6999
annual = annual[(annual['sic'] <= 5999) | (annual['sic'] >= 7000)]
# fill nan values with 0 for R&D
annual['xrd'].fillna(0, inplace=True)
# compute annual market value of equity
annual['mktvaleq'] = annual['csho'] * annual['prcc_f']
# sort by datadate for easier viewing
annual.sort_values('datadate', inplace=True)
# remove dates that are way outside our range (1975-2022 only)
annual = annual[((annual['datadate'] >= '1975-01-01') & (annual['datadate'] <= '2022-01-01'))]
# drop firms without a market value of equity
annual.dropna(subset=['mktvaleq'], inplace=True)
# drop columns no longer needed
annual.drop(columns=['curcd', 'exchg', 'mkvalt', 'prcc_f', 'sic', 'csho', 'fyr'], inplace=True)
# rename datadate to yearly_date for better understanding
annual.rename({'datadate': 'yearly_date'}, axis=1, inplace=True)
# if R&D is negative, set it to 0
annual['xrd'] = annual['xrd'].apply(lambda x: 0 if x < 0 else x)

annual.head(2)

,GVKEY,LPERMNO,yearly_date,fyear,conm,xrd,mktvaleq
137725,10813,50278,1975-01-29,1974,URS CORP,0.0,5.848500
96801,7911,57745,1975-01-29,1974,NOBILITY HOMES INC,0.0,4.229125


# Reconstitution

### Edit 'fyear' to be the year of the reconstitution (April 1st to April 1st)

Most firms had their fiscal year end on December 31st, therefore after lagging the data by 3 months, the fiscal year end was March 31st. Therefore if we reconstitute on April 1st, majority of the firms would have their most recent annuals data available.

In [7]:
# if month is after april (4), then add 1 to the year to get the correct year
annual['year'] = annual['yearly_date'].dt.year
annual['month'] = annual['yearly_date'].dt.month
annual['fyear'] = np.where(annual['month'] >= 4, annual['year'] + 1, annual['year'])

# if month is after april (4), then keep the year to line it up with the correct yearly financial data
monthly['year'] = monthly['monthly_date'].dt.year
monthly['month'] = monthly['monthly_date'].dt.month
monthly['fyear'] = np.where(monthly['month'] >= 4, monthly['year'], monthly['year'] - 1)

### Create rdc/mktvaleq column for splitting our data into R&D quintiles and Non-R&D

In [8]:
# create rdc column by taking a weighted average of R&D over the past 5 years
annual['rdc'] = annual['xrd'] \
                + 0.8 * annual.groupby(['GVKEY'])['xrd'].shift(1).fillna(0) \
                + 0.6 * annual.groupby(['GVKEY'])['xrd'].shift(2).fillna(0) \
                + 0.4 * annual.groupby(['GVKEY'])['xrd'].shift(3).fillna(0) \
                + 0.2 * annual.groupby(['GVKEY'])['xrd'].shift(4).fillna(0)

# compute a column for rdc / mktvaleq (Moving Average R&D / Market Value of Equity)
annual['rdc_mktvaleq'] = annual['rdc'] / annual['mktvaleq']
# drop unnecessary columns
annual = annual[['GVKEY', 'LPERMNO', 'conm', 'yearly_date','fyear', 
        'mktvaleq', 'xrd', 'rdc', 'rdc_mktvaleq']]

annual.head(2)

,GVKEY,LPERMNO,conm,yearly_date,fyear,mktvaleq,xrd,rdc,rdc_mktvaleq
137725,10813,50278,URS CORP,1975-01-29,1975,5.848500,0.0,0.0,0.0
96801,7911,57745,NOBILITY HOMES INC,1975-01-29,1975,4.229125,0.0,0.0,0.0


## Merge Monthly and Annual Data

In [9]:
# merge the monthly and annual data sets on LPERMNO and PERMNO as well as fyear
# inner because we only want data that has both monthly and annual data
data = annual.merge(monthly, how = 'inner', left_on=['LPERMNO', 'fyear'], right_on=['PERMNO', 'fyear'])

# drop unnecessary columns by subsetting by the columns we need
data = data[['GVKEY', 'PERMNO', 'company_name', 'yearly_date', 'fyear', 'monthly_date', 
            'mktvaleq', 'xrd', 'rdc', 'rdc_mktvaleq', 'monthly_price', 'monthly_return', 'shares_outstanding']]

# There are cases where there are multiple financial data points for the same month, so we need to drop those duplicates
# We want to keep the entry where the monthly data is using the most up to date fiscal year financial numbers (Closest to April 1st)
data = data.sort_values('yearly_date').drop_duplicates(['PERMNO', 'monthly_date'], keep='last')

data.head(2)

,GVKEY,PERMNO,company_name,yearly_date,fyear,monthly_date,mktvaleq,xrd,rdc,rdc_mktvaleq,monthly_price,monthly_return,shares_outstanding
0,10813,50278,U R S CORP,1975-01-29,1975,1975-04-30,5.8485,0.0,0.0,0.0,2.625,-0.091357,3342.0
1036,6859,49023,M B P X L CORP,1975-01-29,1975,1975-11-28,13.0645,0.0,0.0,0.0,10.125,0.113141,1786.0


# Three Sample Periods (1981-2012, 1981-1999, 2000-2012)

### Make a dataframe that includes each entries quintile for its year

In [10]:
# new dataframe that will include quintiles, returns for equally and value weighted portfolios
all_quant = pd.DataFrame()

def get_quintiles(df):
    '''
        Parameters:
            df: DataFrame that contains the data for a specific fyear
        Returns:
            quintiles: DataFrame that contains the quintiles for a specific fyear (0 for Non-R&D firms, 1-5 for R&D firms)
    '''

    # split df into R&D firms and Non-R&D firms
    rd = df[df['rdc_mktvaleq'] != 0]
    no_rd = df[df['rdc_mktvaleq'] == 0]

    # get the quintiles for R&D firms
    rd['quintile'] = pd.qcut(rd['rdc_mktvaleq'], 5, labels=[1, 2, 3, 4, 5])
    # assign a quintile of 0 to Non-R&D firms
    no_rd['quintile'] = 0

    # combine r&d and no r&d into one dataframe
    df = pd.concat([rd, no_rd])

    return df

# loop through each fyear to get the quintiles for each entry
for year in data['fyear'].unique():
    # get the data for that fyear
    df = data[data['fyear'] == year]
    # get the quintiles for that fyear
    df = get_quintiles(df)
    # add the data with the quintile splits to the all_quant dataframe
    all_quant = pd.concat([all_quant, df])

### Add columns for computing value weighted returns

In [11]:
# add a column for the monthly market value of equity for each firm (Monthly Price * Shares Outstanding)
all_quant['monthly_mktvaleq'] = all_quant['monthly_price'] * all_quant['shares_outstanding']
# sort the data by PERMNO and monthly_date for easier viewing
all_quant = all_quant.sort_values(['PERMNO', 'monthly_date'])
# add a column for lagged monthly market value of equity (group by firm and quintile and shift by 1 month)
all_quant['lagged_monthly_mktvaleq'] = all_quant.groupby(['PERMNO', 'quintile'])['monthly_mktvaleq'].shift(1)
# drop the nan values from shifting (first month of each firm, most are in 1975 so won't effect results)
all_quant.dropna(subset=['lagged_monthly_mktvaleq'], inplace=True)

# Add a column for the sum of lagged monthly market value of equity (group by month and quintile)
# Transform method is used to give each row it's corrrect sum of lagged monthly market value of equity (by month and quintile)
all_quant['sum_lagged_monthly_mktvaleq'] = all_quant.groupby(['monthly_date', 'quintile'])['lagged_monthly_mktvaleq'].transform('sum')

# add a column with the weight of each firm in the value weighted portfolio
# This will allows us to calculate the value weighted returns which is:
#    (lagged monthly market value of equity / total of lagged monthly market value of equity (by month and quintile)) * monthly return
all_quant['weight'] = all_quant['lagged_monthly_mktvaleq'] / all_quant['sum_lagged_monthly_mktvaleq']

# weighted returns are the monthly returns * the weight for the firm for the month
all_quant['weighted_monthly_return'] = all_quant['monthly_return'] * all_quant['weight']

all_quant.head(2)

,GVKEY,PERMNO,company_name,yearly_date,fyear,monthly_date,mktvaleq,xrd,rdc,rdc_mktvaleq,monthly_price,monthly_return,shares_outstanding,quintile,monthly_mktvaleq,lagged_monthly_mktvaleq,sum_lagged_monthly_mktvaleq,weight,weighted_monthly_return
469066,13007,10000,OPTIMUM MANUFACTURING INC,1987-01-29,1987,1987-05-29,2.88225,0.039,0.039,0.013531,0.21875,-0.070467,3893.0,1,851.59375,912.44134,1.733448e+08,0.000005,-3.709196e-07
458844,12994,10001,GREAT FALLS GAS CO,1986-09-28,1987,1987-05-29,5.91000,0.000,0.000,0.000000,5.68750,-0.075229,991.0,0,5636.31250,6069.87500,9.223191e+08,0.000007,-4.950896e-07


### Function to get the equally weighted returns by month and quintile

In [12]:
def get_equal_weighted(df, start, end):
    '''
        Parameters:
            df: DataFrame with all the quintiles and returns for each firm for each month
            start: The first date in the time period
            end: The last date in the time period
        Returns:
            equal_returns: DataFrame with the equal weighted returns for each quintile in the time period
            monthly_return: DataFrame with the monthly returns for each month per quintile in the time period
    '''

    # filter data for the specific time period
    df = df[(df['monthly_date'] >= start) & (df['monthly_date'] <= end)]

    # get the monthly returns for each month per quintile (mean for equal weighted returns)
    monthly_return = df.groupby(['quintile', 'monthly_date'])['monthly_return'].mean().reset_index()

    # sort the values by quintile and monthly_date for easier viewing when returned
    monthly_return.sort_values(['quintile', 'monthly_date'], inplace=True)

    # get the equal weighted returns for each quintile (mean of monthly returns per quintile)
    equal_returns = monthly_return.groupby('quintile')['monthly_return'].mean().reset_index()
    
    # reformat the monthly_returns column to be in percent
    equal_returns['monthly_return'] = equal_returns['monthly_return'] * 100

    # print start and end date for time period, easier to understand when looking at the results
    print(start, end)
    
    return equal_returns, monthly_return

## Equally Weighted Results (1981-2012, 1981-1999, 2000-2012)

### Equally Weighted Results (1981-2012)

In [13]:
equal_returns_1981_2012, equal_monthly_returns_1981_2012 = get_equal_weighted(all_quant, '1981-07-01', '2012-12-31')
equal_returns_1981_2012

1981-07-01 2012-12-31


,quintile,monthly_return
0,0,0.711636
1,1,0.265736
2,2,0.614688
3,3,0.871093
4,4,1.146084
5,5,1.761361


### Equally Weighted Results (1981-1999)

In [14]:
equal_returns_1981_1999, equal_monthly_returns_1981_1999 = get_equal_weighted(all_quant, '1981-07-01', '1999-12-31')
equal_returns_1981_1999

1981-07-01 1999-12-31


,quintile,monthly_return
0,0,0.572603
1,1,0.244928
2,2,0.623063
3,3,0.872895
4,4,1.132175
5,5,1.708383


### Equally Weighted Results (2000-2012)

In [15]:
equal_returns_2000_2012, equal_monthly_returns_2000_2012 = get_equal_weighted(all_quant, '2000-01-01', '2012-12-31')
equal_returns_2000_2012

2000-01-01 2012-12-31


,quintile,monthly_return
0,0,0.909491
1,1,0.295346
2,2,0.602771
3,3,0.868530
4,4,1.165878
5,5,1.836754


### Equally Weighted Results (1981-2012) per Month

This allows me to create the High - Low column for the regressions by subtracting the low quintile from the high quintile for each month.

In [16]:
# pivot makes each quintile a column and the monthly returns the values
total_equal_1981_2012 = equal_monthly_returns_1981_2012.pivot(index='monthly_date', columns='quintile', values='monthly_return')
# resets the index so that the monthly_date is a column again
total_equal_1981_2012 = total_equal_1981_2012.reset_index()
# create the high - low column by subtracting the low quintile (1) from the high quintile (5)
total_equal_1981_2012['High - Low'] = total_equal_1981_2012[5] - total_equal_1981_2012[1]
total_equal_1981_2012

quintile,monthly_date,0,1,2,3,4,5,High - Low
0,1981-07-31,-0.025439,-0.025538,-0.049509,-0.039951,-0.044144,-0.052400,-0.026862
1,1981-08-31,-0.078088,-0.097306,-0.098097,-0.105482,-0.110519,-0.097209,0.000097
2,1981-09-30,-0.099894,-0.114615,-0.102884,-0.087690,-0.093217,-0.099878,0.014737
3,1981-10-30,0.054894,0.087470,0.072907,0.075323,0.064404,0.056069,-0.031401
4,1981-11-30,0.006888,0.004924,0.005925,0.026594,0.011648,0.006884,0.001960
...,...,...,...,...,...,...,...,...
373,2012-08-31,0.028908,0.028328,0.037038,0.033323,0.014227,0.030937,0.002609
374,2012-09-28,0.042242,0.036323,0.029659,0.046601,0.026806,0.042816,0.006493
375,2012-10-31,-0.012773,-0.015321,-0.016445,-0.021461,-0.044131,-0.062519,-0.047198
376,2012-11-30,-0.004751,0.016876,0.004029,0.005597,0.012600,0.026185,0.009310


### Fama French Data: 1981 - 2012 (Equally Weighted)

In [17]:
# filter fama french data for the time period
fama_french_1981_2012 = fama_french[(fama_french['year_month'] >= '1981-07') & (fama_french['year_month'] <= '2012-12')].reset_index(drop=True)
# add the high - low column to the fama french data to make it easier to run the regressions
fama_french_1981_2012['High - Low'] = total_equal_1981_2012['High - Low']
fama_french_1981_2012.head(2)

,year_month,Mkt-RF,SMB,HML,RF,year,month,High - Low
0,1981-07,-1.54,-2.19,-0.50,0.0124,1981,7,-0.026862
1,1981-08,-7.04,-1.95,4.76,0.0128,1981,8,0.000097


### CAPM Alpha: 1981 - 2012 (Equal Weighted)

In [18]:
# Regress the high - low column on the market risk premium to find the CAPM alpha
X = fama_french_1981_2012['Mkt-RF']
y = fama_french_1981_2012['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('CAPM Alpha for 1981-2012 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for CAPM Alpha for 1981-2012 (Monthly):', round(results.tvalues[0], 4))

CAPM Alpha for 1981-2012 (Monthly): 1.4372%
T-statistic for CAPM Alpha for 1981-2012 (Monthly): 5.691


### Fama French 3 Factors Alpha: 1981 - 2012 (Equal Weighted)

In [19]:
# Regress the high - low column on the market risk premium, SMB and HML to find the FF3 Factor alpha
X = fama_french_1981_2012[['Mkt-RF', 'SMB', 'HML']]
y = fama_french_1981_2012['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('Fama French 3 Factor Alpha for 1981-2012 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for Fama French 3 Factor Alpha for 1981-2012 (Monthly):', round(results.tvalues[0], 4))

Fama French 3 Factor Alpha for 1981-2012 (Monthly): 1.4141%
T-statistic for Fama French 3 Factor Alpha for 1981-2012 (Monthly): 6.3006


### Sharpe Ratio: 1981 - 2012 (Equal Weighted)

In [20]:
# compute the sharpe ratio for the high - low column (mean / standard deviation)
mean_high_low = fama_french_1981_2012['High - Low'].mean()
std_high_low = fama_french_1981_2012['High - Low'].std()

# multiply by sqrt(12) to get the annualized sharpe ratio
annualized_sharpe_ratio = (mean_high_low / std_high_low) * np.sqrt(12)

print('Annualized Sharpe Ratio for 1981-2012 (Annual):', round(annualized_sharpe_ratio, 4))

Annualized Sharpe Ratio for 1981-2012 (Annual): 1.0599


## Value Weighted Results (1981-2012, 1981-1999, 2000-2012)

### Function to get the value weighted returns by month and quintile

In [21]:
def get_value_weighted(df, start, end):
    '''
        Parameters: 
            df: DataFrame with all the quintiles and returns for each firm for each month
            start: The first date in the time period
            end: The last date in the time period
        Returns:
            value_returns: DataFrame with the value weighted returns for each quintile in the time period
            monthly_return: DataFrame with the monthly returns for each month per quintile in the time period
    
    '''
    # filter data for the specific time period
    df = df[(df['monthly_date'] >= start) & (df['monthly_date'] <= end)]

    # get the monthly returns for each month per quintile (sum of weighted monthly return) 
    monthly_return = df.groupby(['quintile', 'monthly_date'])['weighted_monthly_return'].sum().reset_index()

    # get the value weighted returns for each quintile (mean of weighted monthly return per quintile)
    # its the mean because the weighted monthly returns are the same for each quintile
    value_returns = monthly_return.groupby('quintile')['weighted_monthly_return'].mean().reset_index()

    # reformat the weighted_monthly_returns column to be in percent
    value_returns['weighted_monthly_return'] = value_returns['weighted_monthly_return'] * 100

    # print start and end date for time period, easier to understand when looking at the results
    print(start, end)
    return value_returns, monthly_return


### Value Weighted Results (1981-2012)

In [22]:
value_return_1981_2012, value_monthly_returns_1981_2012 = get_value_weighted(all_quant, '1981-07-01', '2012-12-31')
value_return_1981_2012

1981-07-01 2012-12-31


,quintile,weighted_monthly_return
0,0,0.551550
1,1,0.458164
2,2,0.615012
3,3,0.687071
4,4,0.898108
5,5,0.991629


### Value Weighted Results (1981-1999)

In [23]:
value_return_1981_1999, value_monthly_returns_1981_1999 = get_value_weighted(all_quant, '1981-07-01', '1999-12-31')
value_return_1981_1999

1981-07-01 1999-12-31


,quintile,weighted_monthly_return
0,0,0.764954
1,1,0.698426
2,2,0.995817
3,3,1.084824
4,4,1.081853
5,5,1.181369


### Value Weighted Results (2000-2012)

In [24]:
value_return_2000_2012, value_monthly_returns_2000_2012 = get_value_weighted(all_quant, '2000-01-01', '2012-12-31')
value_return_2000_2012

2000-01-01 2012-12-31


,quintile,weighted_monthly_return
0,0,0.247862
1,1,0.116252
2,2,0.073097
3,3,0.121039
4,4,0.636626
5,5,0.721614


### Value Weighted Results (1981-2012) per Month 

This allows me to create the High - Low column for the regressions by subtracting the low quintile from the high quintile for each month.

In [25]:
# pivot makes each quintile a column and the monthly returns the values
total_value_1981_2012 = value_monthly_returns_1981_2012.pivot(index='monthly_date', columns='quintile', values='weighted_monthly_return')
# resets the index so that the monthly_date is a column again
total_value_1981_2012 = total_value_1981_2012.reset_index()
# create the high - low column by subtracting the low quintile (1) from the high quintile (5)
total_value_1981_2012['High - Low'] = total_value_1981_2012[5] - total_value_1981_2012[1]
total_value_1981_2012

quintile,monthly_date,0,1,2,3,4,5,High - Low
0,1981-07-31,-0.008123,0.055956,0.005565,-0.032474,-0.040559,-0.056509,-0.112465
1,1981-08-31,-0.070968,-0.080284,-0.050008,-0.083597,-0.070485,-0.105695,-0.025411
2,1981-09-30,-0.081258,-0.128556,-0.061268,-0.061388,-0.048509,-0.098185,0.030372
3,1981-10-30,0.065795,0.090411,0.037909,0.048415,0.008460,-0.026369,-0.116780
4,1981-11-30,0.028501,0.050888,0.033051,0.018046,0.037902,0.015492,-0.035396
...,...,...,...,...,...,...,...,...
373,2012-08-31,0.017398,0.031005,0.034971,0.022328,0.010996,0.045340,0.014335
374,2012-09-28,0.029683,0.020950,0.038247,0.021840,0.008594,0.031814,0.010864
375,2012-10-31,-0.007529,-0.036594,-0.032710,-0.028263,-0.029340,0.009392,0.045986
376,2012-11-30,-0.003732,0.003307,0.024531,0.017026,0.016352,0.034663,0.031356


### Fama French Data: 1981 - 2012 (Value Weighted)

In [26]:
# filter fama french data for the time period
fama_french_1981_2012 = fama_french[(fama_french['year_month'] >= '1981-07') & (fama_french['year_month'] <= '2012-12')].reset_index(drop=True)
# add the high - low column to the fama french data to make it easier to run the regressions
fama_french_1981_2012['High - Low'] = total_value_1981_2012['High - Low']
fama_french_1981_2012.head(2)

,year_month,Mkt-RF,SMB,HML,RF,year,month,High - Low
0,1981-07,-1.54,-2.19,-0.50,0.0124,1981,7,-0.112465
1,1981-08,-7.04,-1.95,4.76,0.0128,1981,8,-0.025411


### CAPM Alpha: 1981 - 2012 (Value Weighted)

In [27]:
# Regress the high - low column on the market risk premium to find the CAPM alpha
X = fama_french_1981_2012['Mkt-RF']
y = fama_french_1981_2012['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('CAPM Alpha for 1981-2012 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for CAPM Alpha for 1981-2012 (Monthly):', round(results.tvalues[0], 4))

CAPM Alpha for 1981-2012 (Monthly): 0.333%
T-statistic for CAPM Alpha for 1981-2012 (Monthly): 1.3365


### Fama French 3 Factors Alpha: 1981 - 2012 (Value Weighted)

In [28]:
# Regress the high - low column on the market risk premium, SMB and HML to find the FF3 Factor alpha
X = fama_french_1981_2012[['Mkt-RF', 'SMB', 'HML']]
y = fama_french_1981_2012['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('Fama French 3 Factor Alpha for 1981-2012 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for Fama French 3 Factor Alpha for 1981-2012 (Monthly):', round(results.tvalues[0], 4))

Fama French 3 Factor Alpha for 1981-2012 (Monthly): 0.1094%
T-statistic for Fama French 3 Factor Alpha for 1981-2012 (Monthly): 0.4943


### Sharpe Ratio: 1981 - 2012 (Value Weighted)

In [29]:
# compute the sharpe ratio for the high - low column (mean / standard deviation)
mean_high_low = fama_french_1981_2012['High - Low'].mean()
std_high_low = fama_french_1981_2012['High - Low'].std()

# multiply by sqrt(12) to get the annualized sharpe ratio
annualized_sharpe_ratio = (mean_high_low / std_high_low) * np.sqrt(12)

print('Annualized Sharpe Ratio for 1981-2012 (Annual):', round(annualized_sharpe_ratio, 4))

Annualized Sharpe Ratio for 1981-2012 (Annual): 0.365


# Drop Top 1000 Largest Firms Each Year (1981-2012, 1981-1999, 2000-2012)

In [30]:
# new dataframe that will include quintiles, returns for equally and value weighted portfolios
all_quant_1000 = all_quant.copy()

# group by fyear and permnno to get each firms yearly market value of equity
# mean because the market value of equity is the same for the whole fyear
top_1000 = all_quant.groupby(['fyear', 'PERMNO'])['mktvaleq'].mean().reset_index()
# sort values for easier viewing
top_1000.sort_values('mktvaleq', ascending=False)
# get the top 1000 permnos for each year by market value of equity
top_1000 = top_1000.sort_values(['fyear', 'mktvaleq'], ascending=False).groupby('fyear').head(1000)

# drop the top 1000 larget firms each year
for year in top_1000['fyear'].unique():
    # get the df with only the firms for that year
    df = top_1000[top_1000['fyear'] == year]
    # create a filter (mask) where the PERMNO is in the top 1000 firms for the year
    mask = all_quant['PERMNO'].isin(df['PERMNO']) & (all_quant['fyear'] == year)
    # remove the top 1000 largest firms for the year
    all_quant_1000 = all_quant_1000[~mask]

# new dataframe that will include quintiles, returns for equally and value weighted portfolios
new_all_quant = pd.DataFrame()

# loop through each year and create quintiles for each year
for year in all_quant_1000['fyear'].unique():
    # get the df for the year
    df = all_quant_1000[all_quant_1000['fyear'] == year]
    # create quintiles for the year using the function
    df = get_quintiles(df)
    # append the df for the year with quintiles to the new_all_quant df
    new_all_quant = pd.concat([new_all_quant, df])

### Add columns for computing value weighted returns

In [31]:
# add column for monthly market value of equity (Monthly Price * Shares Outstanding)
new_all_quant['monthly_mktvaleq'] = new_all_quant['monthly_price'] * new_all_quant['shares_outstanding']
# sort the values by date and permno for easier viewing
new_all_quant = new_all_quant.sort_values(['PERMNO', 'monthly_date'])
# add a column for lagged monthly market value of equity (group by firm and quintile and shift by 1 month)
new_all_quant['lagged_monthly_mktvaleq'] = new_all_quant.groupby(['PERMNO', 'quintile'])['monthly_mktvaleq'].shift(1)
# drop the nan values from shifting (first month of each firm, most are in 1975 so won't effect results)
new_all_quant.dropna(subset=['lagged_monthly_mktvaleq'], inplace=True)

# Add a column for the sum of lagged monthly market value of equity (group by month and quintile)
# Transform method is used to give each row it's corrrect sum of lagged monthly market value of equity (by month and quintile)
new_all_quant['sum_lagged_monthly_mktvaleq'] = new_all_quant.groupby(['monthly_date', 'quintile'])['lagged_monthly_mktvaleq'].transform('sum')

# add a column with the weight of each firm in the value weighted portfolio
# This will allows us to calculate the value weighted returns which is:
#    (lagged monthly market value of equity / total of lagged monthly market value of equity (by month and quintile)) * monthly return
new_all_quant['weight'] = new_all_quant['lagged_monthly_mktvaleq'] / new_all_quant['sum_lagged_monthly_mktvaleq']

# weighted returns are the monthly returns * the weight for the firm for the month
new_all_quant['weighted_monthly_return'] = new_all_quant['monthly_return'] * new_all_quant['weight']

new_all_quant.head(2)

,GVKEY,PERMNO,company_name,yearly_date,fyear,monthly_date,mktvaleq,xrd,rdc,rdc_mktvaleq,monthly_price,monthly_return,shares_outstanding,quintile,monthly_mktvaleq,lagged_monthly_mktvaleq,sum_lagged_monthly_mktvaleq,weight,weighted_monthly_return
458845,12994,10001,GREAT FALLS GAS CO,1986-09-28,1987,1987-06-30,5.91,0.0,0.0,0.0,5.875,0.046629,991.0,0,5822.125,5636.3125,7.977917e+07,0.000071,0.000003
458846,12994,10001,GREAT FALLS GAS CO,1986-09-28,1987,1987-07-31,5.91,0.0,0.0,0.0,6.000,0.016677,991.0,0,5946.000,5822.1250,8.392279e+07,0.000069,0.000001


### Value Weighted Drop 1000 Largest Results (1981-2012)

In [32]:
value_return_1000_1981_2012, value_monthly_returns_1000_1981_2012 = get_value_weighted(new_all_quant, '1981-07-01', '2012-12-31')
value_return_1000_1981_2012

1981-07-01 2012-12-31


,quintile,weighted_monthly_return
0,0,0.595850
1,1,0.399206
2,2,0.584992
3,3,0.743091
4,4,0.923087
5,5,1.305694


### Value Weighted Drop 1000 Largest Results (1981-1999)

In [33]:
value_return_1000_1981_1999, value_monthly_returns_1000_1981_1999 = get_value_weighted(new_all_quant, '1981-07-01', '1999-12-31')
value_return_1000_1981_1999

1981-07-01 1999-12-31


,quintile,weighted_monthly_return
0,0,0.498790
1,1,0.442720
2,2,0.661416
3,3,0.848870
4,4,1.032622
5,5,1.288191


### Value Weighted Drop 1000 Largest Results (2000-2012)

In [34]:
value_return_1000_2000_2012, value_monthly_returns_1000_2000_2012 = get_value_weighted(new_all_quant, '2000-01-01', '2012-12-31')
value_return_1000_2000_2012

2000-01-01 2012-12-31


,quintile,weighted_monthly_return
0,0,0.733974
1,1,0.337282
2,2,0.476237
3,3,0.592558
4,4,0.767209
5,5,1.330601


### Value Weighted Drop 1000 Largest Results (1981-2012) per Month

This allows me to create the High - Low column for the regressions by subtracting the low quintile from the high quintile for each month.

In [35]:
# pivot the df so that the columns are the quintiles and the values are the monthly returns
total_value_1000_1981_2012 = value_monthly_returns_1000_1981_2012.pivot(index='monthly_date', columns='quintile', values='weighted_monthly_return')
# reset index so that the monthly_date is a column
total_value_1000_1981_2012 = total_value_1000_1981_2012.reset_index()
# add a column for high - low, which is the difference between the highest and lowest quintile for R&D
total_value_1000_1981_2012['High - Low'] = total_value_1000_1981_2012[5] - total_value_1000_1981_2012[1]
total_value_1000_1981_2012

quintile,monthly_date,0,1,2,3,4,5,High - Low
0,1981-07-31,-0.033569,-0.047503,-0.068878,-0.037728,-0.051461,-0.036836,0.010667
1,1981-08-31,-0.086396,-0.116843,-0.130441,-0.121269,-0.132399,-0.109261,0.007582
2,1981-09-30,-0.096905,-0.124186,-0.104069,-0.072961,-0.106890,-0.100469,0.023717
3,1981-10-30,0.067668,0.109050,0.093944,0.116231,0.078220,0.087660,-0.021390
4,1981-11-30,0.010812,0.004715,-0.001379,0.016452,0.005636,0.006681,0.001965
...,...,...,...,...,...,...,...,...
373,2012-08-31,0.040056,0.026005,0.047969,0.029974,0.006234,0.033192,0.007186
374,2012-09-28,0.041536,0.026259,0.036998,0.020694,0.017750,0.034548,0.008289
375,2012-10-31,-0.009779,-0.021499,-0.019432,-0.043225,-0.058060,-0.091068,-0.069569
376,2012-11-30,0.001364,0.010056,0.015576,0.010607,0.029274,0.048198,0.038142


### Fama French Data: 1981 - 2012 (Value Weighted Drop 1000 Largest)

In [36]:
# filter the fama french data to only include the time period
fama_french_1981_2012 = fama_french[(fama_french['year_month'] >= '1981-07') & (fama_french['year_month'] <= '2012-12')].reset_index(drop=True)
# add a column for the high - low to make running the regression easier
fama_french_1981_2012['High - Low'] = total_value_1000_1981_2012['High - Low']
fama_french_1981_2012.head(2)

,year_month,Mkt-RF,SMB,HML,RF,year,month,High - Low
0,1981-07,-1.54,-2.19,-0.50,0.0124,1981,7,0.010667
1,1981-08,-7.04,-1.95,4.76,0.0128,1981,8,0.007582


### CAPM Alpha: 1981 - 2012 (Value Weighted Drop 1000 Largest)

In [37]:
# Regress High - Low on the Market Risk Premium to find CAPM alpha
X = fama_french_1981_2012['Mkt-RF']
y = fama_french_1981_2012['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('CAPM Alpha for 1981-2012 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for CAPM Alpha for 1981-2012 (Monthly):', round(results.tvalues[0], 4))

CAPM Alpha for 1981-2012 (Monthly): 0.7821%
T-statistic for CAPM Alpha for 1981-2012 (Monthly): 3.5173


### Fama French 3 Factors Alpha: 1981 - 2012 (Value Weighted Drop 1000 Largest)

In [38]:
# Regress High - Low on the Market Risk Premium, SMB, and HML to find FF3 alpha
X = fama_french_1981_2012[['Mkt-RF', 'SMB', 'HML']]
y = fama_french_1981_2012['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('Fama French 3 Factor Alpha for 1981-2012 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for Fama French 3 Factor Alpha for 1981-2012 (Monthly):', round(results.tvalues[0], 4))

Fama French 3 Factor Alpha for 1981-2012 (Monthly): 0.7559%
T-statistic for Fama French 3 Factor Alpha for 1981-2012 (Monthly): 3.5429


### Sharpe Ratio: 1981 - 2012 (Value Weighted Drop 1000 Largest)

In [39]:
# sharpe ratio is mean / standard deviation of high - low
mean_high_low = fama_french_1981_2012['High - Low'].mean()
std_high_low = fama_french_1981_2012['High - Low'].std()

# annualize the sharpe ratio by sqrt(12)
annualized_sharpe_ratio = (mean_high_low / std_high_low) * np.sqrt(12)

print('Annualized Sharpe Ratio for 1981-2012 (Annual):', round(annualized_sharpe_ratio, 4))

Annualized Sharpe Ratio for 1981-2012 (Annual): 0.7139


# From 1981 - 2021

## Equally Weighted

### Equally Weighted Results (1981-2021)

In [40]:
equal_returns_1981_2021, equal_monthly_returns_1981_2021 = get_equal_weighted(all_quant, '1981-07-01', '2021-12-31')
equal_returns_1981_2021

1981-07-01 2021-12-31


,quintile,monthly_return
0,0,0.767427
1,1,0.416657
2,2,0.739009
3,3,1.037016
4,4,1.207467
5,5,1.718040


### Equally Weighted Results (1981-2021) per Month

In [41]:
# pivot the df so that the columns are the quintiles and the values are the monthly returns
total_equal_1981_2021 = equal_monthly_returns_1981_2021.pivot(index='monthly_date', columns='quintile', values='monthly_return')
# reset index so that the monthly_date is a column
total_equal_1981_2021 = total_equal_1981_2021.reset_index()
# add a column for high - low, which is the difference between the highest and lowest quintile for R&D
total_equal_1981_2021['High - Low'] = total_equal_1981_2021[5] - total_equal_1981_2021[1]
total_equal_1981_2021

quintile,monthly_date,0,1,2,3,4,5,High - Low
0,1981-07-31,-0.025439,-0.025538,-0.049509,-0.039951,-0.044144,-0.052400,-0.026862
1,1981-08-31,-0.078088,-0.097306,-0.098097,-0.105482,-0.110519,-0.097209,0.000097
2,1981-09-30,-0.099894,-0.114615,-0.102884,-0.087690,-0.093217,-0.099878,0.014737
3,1981-10-30,0.054894,0.087470,0.072907,0.075323,0.064404,0.056069,-0.031401
4,1981-11-30,0.006888,0.004924,0.005925,0.026594,0.011648,0.006884,0.001960
...,...,...,...,...,...,...,...,...
481,2021-08-31,0.014609,0.006645,0.014097,0.033002,0.026331,0.050104,0.043459
482,2021-09-30,-0.012520,-0.062265,-0.052261,-0.051529,-0.046459,-0.044011,0.018254
483,2021-10-29,0.032112,0.035545,0.030196,0.011874,0.014948,-0.038728,-0.074273
484,2021-11-30,-0.050321,-0.070381,-0.068706,-0.063764,-0.073499,-0.112590,-0.042209


### Fama French Data: 1981 - 2021 (Equally Weighted)

In [42]:
# filter the fama french data to only include the time period
fama_french_1981_2021 = fama_french[(fama_french['year_month'] >= '1981-07') & (fama_french['year_month'] <= '2021-12')].reset_index(drop=True)
# add a column for the high - low to make running the regression easier
fama_french_1981_2021['High - Low'] = total_equal_1981_2021['High - Low']
fama_french_1981_2021.head(2)

,year_month,Mkt-RF,SMB,HML,RF,year,month,High - Low
0,1981-07,-1.54,-2.19,-0.50,0.0124,1981,7,-0.026862
1,1981-08,-7.04,-1.95,4.76,0.0128,1981,8,0.000097


### CAPM Alpha: 1981 - 2021 (Equal Weighted)

In [43]:
# Regress High - Low on the Market Risk Premium to find CAPM alpha
X = fama_french_1981_2021['Mkt-RF']
y = fama_french_1981_2021['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('CAPM Alpha for 1981-2021 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for CAPM Alpha for 1981-2021 (Monthly):', round(results.tvalues[0], 4))

CAPM Alpha for 1981-2021 (Monthly): 1.2084%
T-statistic for CAPM Alpha for 1981-2021 (Monthly): 5.3672


### Fama French 3 Factors Alpha: 1981 - 2021 (Equal Weighted)

In [44]:
# Regress High - Low on the Market Risk Premium, SMB, and HML to find FF3 alpha
X = fama_french_1981_2021[['Mkt-RF', 'SMB', 'HML']]
y = fama_french_1981_2021['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('Fama French 3 Factor Alpha for 1981-2021 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for Fama French 3 Factor Alpha for 1981-2021 (Monthly):', round(results.tvalues[0], 4))

Fama French 3 Factor Alpha for 1981-2021 (Monthly): 1.246%
T-statistic for Fama French 3 Factor Alpha for 1981-2021 (Monthly): 6.2535


### Sharpe Ratio: 1981 - 2021 (Equal Weighted)

In [45]:
# sharpe ratio is mean / standard deviation of high - low
mean_high_low = fama_french_1981_2021['High - Low'].mean()
std_high_low = fama_french_1981_2021['High - Low'].std()

# annualize the sharpe ratio by sqrt(12)
annualized_sharpe_ratio = (mean_high_low / std_high_low) * np.sqrt(12)

print('Annualized Sharpe Ratio for 1981-2021 (Annual):', round(annualized_sharpe_ratio, 4))

Annualized Sharpe Ratio for 1981-2021 (Annual): 0.9155


## Value Weighted

### Value Weighted Results (1981-2021)

In [46]:
value_return_1981_2021, value_monthly_returns_1981_2021 = get_value_weighted(all_quant, '1981-07-01', '2021-12-31')
value_return_1981_2021

1981-07-01 2021-12-31


,quintile,weighted_monthly_return
0,0,0.630619
1,1,0.569966
2,2,0.836068
3,3,0.876222
4,4,1.068864
5,5,1.146739


### Value Weighted Results (1981-2021) per Month

In [47]:
# pivot the df so that the columns are the quintiles and the values are the monthly returns
total_value_1981_2021 = value_monthly_returns_1981_2021.pivot(index='monthly_date', columns='quintile', values='weighted_monthly_return')
# reset index so that the monthly_date is a column
total_value_1981_2021 = total_value_1981_2021.reset_index()
# add a column for high - low, which is the difference between the highest and lowest quintile for R&D
total_value_1981_2021['High - Low'] = total_value_1981_2021[5] - total_value_1981_2021[1]
total_value_1981_2021

quintile,monthly_date,0,1,2,3,4,5,High - Low
0,1981-07-31,-0.008123,0.055956,0.005565,-0.032474,-0.040559,-0.056509,-0.112465
1,1981-08-31,-0.070968,-0.080284,-0.050008,-0.083597,-0.070485,-0.105695,-0.025411
2,1981-09-30,-0.081258,-0.128556,-0.061268,-0.061388,-0.048509,-0.098185,0.030372
3,1981-10-30,0.065795,0.090411,0.037909,0.048415,0.008460,-0.026369,-0.116780
4,1981-11-30,0.028501,0.050888,0.033051,0.018046,0.037902,0.015492,-0.035396
...,...,...,...,...,...,...,...,...
481,2021-08-31,0.013034,0.028939,0.037388,0.033545,0.027570,0.004499,-0.024440
482,2021-09-30,-0.038373,-0.036226,-0.056046,-0.059273,-0.056799,-0.010286,0.025940
483,2021-10-29,0.053044,0.099992,0.087137,0.039244,0.028592,0.012471,-0.087521
484,2021-11-30,-0.026716,-0.038406,0.014209,-0.017352,0.031850,0.002533,0.040939


### Fama French Data: 1981 - 2021 (Value Weighted)

In [48]:
# filter the fama french data to only include the time period
fama_french_1981_2021 = fama_french[(fama_french['year_month'] >= '1981-07') & (fama_french['year_month'] <= '2021-12')].reset_index(drop=True)
# add a column for the high - low to make running the regression easier
fama_french_1981_2021['High - Low'] = total_value_1981_2021['High - Low']
fama_french_1981_2021.head(2)

,year_month,Mkt-RF,SMB,HML,RF,year,month,High - Low
0,1981-07,-1.54,-2.19,-0.50,0.0124,1981,7,-0.112465
1,1981-08,-7.04,-1.95,4.76,0.0128,1981,8,-0.025411


### CAPM Alpha: 1981 - 2021 (Value Weighted)

In [49]:
# Regress High - Low on the Market Risk Premium to find CAPM alpha
X = fama_french_1981_2021['Mkt-RF']
y = fama_french_1981_2021['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('CAPM Alpha for 1981-2021 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for CAPM Alpha for 1981-2021 (Monthly):', round(results.tvalues[0], 4))

CAPM Alpha for 1981-2021 (Monthly): 0.3161%
T-statistic for CAPM Alpha for 1981-2021 (Monthly): 1.4668


### Fama French 3 Factors Alpha: 1981 - 2021 (Value Weighted)

In [50]:
# Regress High - Low on the Market Risk Premium, SMB, and HML to find FF3 alpha
X = fama_french_1981_2021[['Mkt-RF', 'SMB', 'HML']]
y = fama_french_1981_2021['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('Fama French 3 Factor Alpha for 1981-2021 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for Fama French 3 Factor Alpha for 1981-2021 (Monthly):', round(results.tvalues[0], 4))

Fama French 3 Factor Alpha for 1981-2021 (Monthly): 0.2166%
T-statistic for Fama French 3 Factor Alpha for 1981-2021 (Monthly): 1.1605


### Sharpe Ratio: 1981 - 2021 (Value Weighted)

In [51]:
# sharpe ratio is mean / standard deviation of high - low
mean_high_low = fama_french_1981_2021['High - Low'].mean()
std_high_low = fama_french_1981_2021['High - Low'].std()

# annualize the sharpe ratio by sqrt(12)
annualized_sharpe_ratio = (mean_high_low / std_high_low) * np.sqrt(12)

print('Annualized Sharpe Ratio for 1981-2021 (Annual):', round(annualized_sharpe_ratio, 4))

Annualized Sharpe Ratio for 1981-2021 (Annual): 0.4043


# From 1981 - 2021 Drop Top 1000 Largest Firms

## Value Weighted

### Value Weighted Drop 1000 Largest Results (1981-2021)

In [52]:
value_return_1981_2021, value_monthly_returns_1981_2021 = get_value_weighted(new_all_quant, '1981-07-01', '2021-12-31')
value_return_1981_2021

1981-07-01 2021-12-31


,quintile,weighted_monthly_return
0,0,0.624803
1,1,0.552197
2,2,0.749625
3,3,0.897403
4,4,1.117715
5,5,1.467367


### Value Weighted Drop 1000 Largest Results (1981-2021) per Month

In [53]:
# pivot the df so that the columns are the quintiles and the values are the monthly returns
total_value_1981_2021 = value_monthly_returns_1981_2021.pivot(index='monthly_date', columns='quintile', values='weighted_monthly_return')
# reset index so that the monthly_date is a column
total_value_1981_2021 = total_value_1981_2021.reset_index()
# add a column for high - low, which is the difference between the highest and lowest quintile for R&D
total_value_1981_2021['High - Low'] = total_value_1981_2021[5] - total_value_1981_2021[1]
total_value_1981_2021

quintile,monthly_date,0,1,2,3,4,5,High - Low
0,1981-07-31,-0.033569,-0.047503,-0.068878,-0.037728,-0.051461,-0.036836,0.010667
1,1981-08-31,-0.086396,-0.116843,-0.130441,-0.121269,-0.132399,-0.109261,0.007582
2,1981-09-30,-0.096905,-0.124186,-0.104069,-0.072961,-0.106890,-0.100469,0.023717
3,1981-10-30,0.067668,0.109050,0.093944,0.116231,0.078220,0.087660,-0.021390
4,1981-11-30,0.010812,0.004715,-0.001379,0.016452,0.005636,0.006681,0.001965
...,...,...,...,...,...,...,...,...
481,2021-08-31,0.007627,0.027059,0.002978,0.050786,0.052188,0.030488,0.003429
482,2021-09-30,-0.012962,-0.046116,-0.041413,-0.051971,-0.038712,-0.030288,0.015828
483,2021-10-29,0.052214,0.030453,0.034145,0.012998,0.023813,-0.018791,-0.049245
484,2021-11-30,-0.020124,-0.065038,-0.044169,-0.045285,-0.058538,-0.049677,0.015362


### Fama French Data: 1981 - 2012 (Value Weighted Drop 1000 Largest)

In [54]:
# filter the fama french data to only include the time period
fama_french_1981_2021 = fama_french[(fama_french['year_month'] >= '1981-07') & (fama_french['year_month'] <= '2021-12')].reset_index(drop=True)
# add a column for the high - low to make running the regression easier
fama_french_1981_2021['High - Low'] = total_value_1981_2021['High - Low']
fama_french_1981_2021.head(2)

,year_month,Mkt-RF,SMB,HML,RF,year,month,High - Low
0,1981-07,-1.54,-2.19,-0.50,0.0124,1981,7,0.010667
1,1981-08,-7.04,-1.95,4.76,0.0128,1981,8,0.007582


### CAPM Alpha: 1981 - 2021 (Value Weighted Drop 1000 Largest)

In [55]:
# Regress High - Low on the Market Risk Premium to find CAPM alpha
X = fama_french_1981_2021['Mkt-RF']
y = fama_french_1981_2021['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('CAPM Alpha for 1981-2021 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for CAPM Alpha for 1981-2021 (Monthly):', round(results.tvalues[0], 4))

CAPM Alpha for 1981-2021 (Monthly): 0.7478%
T-statistic for CAPM Alpha for 1981-2021 (Monthly): 3.6852


### Fama French 3 Factors Alpha: 1981 - 2021 (Value Weighted Drop 1000 Largest)

In [56]:
# Regress High - Low on the Market Risk Premium, SMB, and HML to find FF3 alpha
X = fama_french_1981_2021[['Mkt-RF', 'SMB', 'HML']]
y = fama_french_1981_2021['High - Low']

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print('Fama French 3 Factor Alpha for 1981-2021 (Monthly):', str(round(results.params[0] * 100, 4)) + '%')
print('T-statistic for Fama French 3 Factor Alpha for 1981-2021 (Monthly):', round(results.tvalues[0], 4))

Fama French 3 Factor Alpha for 1981-2021 (Monthly): 0.7654%
T-statistic for Fama French 3 Factor Alpha for 1981-2021 (Monthly): 3.9555


### Sharpe Ratio: 1981 - 2021 (Value Weighted 1000)

In [57]:
# Sharpe ratio is mean / standard deviation of high - low
mean_high_low = fama_french_1981_2021['High - Low'].mean()
std_high_low = fama_french_1981_2021['High - Low'].std()

# annualize the sharpe ratio by sqrt(12)
annualized_sharpe_ratio = (mean_high_low / std_high_low) * np.sqrt(12)

print('Annualized Sharpe Ratio for 1981-2021 (Annual):', round(annualized_sharpe_ratio, 4))

Annualized Sharpe Ratio for 1981-2021 (Annual): 0.7007
